## Setup

In [4]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2015, day=17)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [5]:
# Import performance checking utility
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_example, time_solution

## Part a

### Basic iterative solution
Another combinatorial optimization problem. Let's start with a naive iterative solution.

In [9]:
# Imports
from itertools import combinations

In [6]:
# Functions
def solve_a_basic(input_data: str, *, total_liters: int = 150) -> int:
    """Iterate over all n-tuple combinations for n=1..len(containers) and count those that sum to total_liters."""
    container_capacities = [*map(int, input_data.splitlines())]
    return sum(
        1
        for n in range(1, len(container_capacities) + 1)
        for combo in combinations(container_capacities, n)
        if sum(combo) == total_liters
    )

In [ ]:
# Correctness check
check_example(solve_a_basic, example, total_liters=25)

solve_a found answer 4, which is the correct solution for part A!


In [10]:
# Performance check
time_a_iterative_basic = time_solution(solve_a_basic, input_data, iterations=10)

solve_a_basic takes 95.56 ms


### Optimized iterative solution
Going through all combinations works, but we can optimize by skipping combinations that clearly too small or too large by sorting the containers first.

In [ ]:
# Functions
def trim_combos(
    combos: list[tuple[int, ...]], total_liters: int, max_container_size: int, n_checks: int = 20
) -> list[tuple[int, ...]]:
    """Trim combinations to only those that could reasonably sum to total_liters."""
    new_start, new_end = 0, len(combos)
    step_size = len(combos) // n_checks
    for i in range(step_size, len(combos), step_size):
        # The 1.5 and 2.5 margin factors were empirically determined to
        # give good performance without missing valid combos
        if sum(combos[i]) < (total_liters - 1.5 * max_container_size):
            # Combination too small, skip ahead
            new_start = i
            continue
        if sum(combos[i]) > (total_liters + 2.5 * max_container_size):
            # Combination too large, skip the rest
            new_end = i
            break

    # Trim combos to only those in the valid range
    return combos[new_start:new_end]


def solve_a_optimized(
    input_data: str, *, total_liters: int = 150, n_checks: int = 20, min_containers_only: bool = False
) -> int:
    """Find all valid container combinations but skip sets that are obviously too small or too large."""
    container_capacities = sorted([*map(int, input_data.splitlines())])

    # Find the maximum container sizes for optimization checks
    max_container = container_capacities[-1]

    # For part B: this will hold counts of combinations by number of containers used
    combo_counts = [0] * (len(container_capacities))

    for n in range(1, len(container_capacities) + 1):
        if sum(container_capacities[-n:]) < total_liters:
            # Largest n containers still less than total_liters, continue to next n
            continue
        if sum(container_capacities[:n]) > total_liters:
            # Smallest n containers already exceed total_liters, we can stop searching
            break

        combos = [*combinations(container_capacities, n)]

        # Check a sample of combinations to skip those that are obviously too small or too large
        if len(combos) > 100:
            combos = trim_combos(combos, total_liters, max_container, n_checks=n_checks)

        for combo in combos:
            if sum(combo) == total_liters:
                combo_counts[n] += 1

    if min_containers_only:
        # Find the minimum number of containers that can reach total_liters
        for combo_count in combo_counts:
            if combo_count > 0:
                return combo_count

    return sum(combo_counts)

In [12]:
# Correctness check
check_example(solve_a_optimized, example, total_liters=25)

solve_a_optimized found answer 4, which is the correct solution for part A!


In [13]:
# Performance check
time_a_iterative_optimized = time_solution(solve_a_optimized, input_data, iterations=10)
print(f"This is {time_a_iterative_basic / time_a_iterative_optimized:.2f}x faster than the basic solution.")

solve_a_optimized takes 59.67 ms
This is 1.60x faster than the basic solution.


### Dynamic programming solution
Actually, on closer inspection, this looks like a classic knapsack problem that can be solved with dynamic programming. This way, we count the number of unique ways that we can reach the target liter amount, recursively building up from smaller amounts.

In [212]:
def solve_a_dp(input_data: str, *, total_liters: int = 150) -> int:
    """Count the number of distinct container combinations that sum to total_liters using dynamic programming."""
    containers = [int(line) for line in input_data.splitlines()]

    # combo_count[l]: number of combinations that sum to l liters
    combo_count = [1] + [0] * (total_liters)

    # Process each container once. Iterate subsets backwards to avoid reusing the same container.
    for c in containers:
        if c > total_liters:
            # Skip containers larger than the target
            continue
        for subset_l in range(total_liters, c - 1, -1):
            # Every subset that made (subset_l - c) liters can be extended with c to make subset_l liters
            combo_count[subset_l] += combo_count[subset_l - c]

    # Return the number of combinations that sum to total_liters
    return combo_count[total_liters]

In [205]:
# Correctness check
check_example(solve_a_dp, example, total_liters=25)

solve_a_dp found answer 4, which is the correct solution for part A!


In [ ]:
# Performance check
time_a_dp = time_solution(solve_a_dp, input_data, time_unit="us")
print(
    f"This is {1000 * time_a_iterative_basic / time_a_dp:.0f}x faster than the basic solution "
    f"and {1000 * time_a_iterative_optimized / time_a_dp:.0f}x faster than the optimized combinatorial solution."
)

solve_a_dp takes 62.89 μs
This is 1494x faster than the basic solution and 601x faster than the optimized combinatorial solution.


In [213]:
# Submit answer
puzzle.answer_a = solve_a_dp(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine. [Continue to Part Two]


## Part b

### Iterative approach
Same as part a, but now we need to find the minimum number of containers that can reach the target amount, and then count how many combinations use that many containers.

I've just adapted the previous iterative solution a bit, performance is not affected.

In [223]:
# Performance check
time_b_iterative = time_solution(solve_a_optimized, input_data, iterations=10, min_containers_only=True)

solve_a_optimized takes 38.15 ms


### Dynamic programming approach
We can also adapt the dynamic programming solution to keep track of the number of containers used for each liter amount. Note that this is quite a lot slower than the DP solution for part A, as we now need to keep track of more states.

In [218]:
def solve_b_dp(input_data: str, *, total_liters: int = 150) -> int:
    """Return number of combinations that use the minimum number of containers to reach total_liters."""
    containers = [int(line) for line in input_data.splitlines()]
    n = len(containers)

    # combo_count[k][s]: amount of ways to store s liters using exactly k containers
    combo_count = [[0] * (total_liters + 1) for _ in range(n + 1)]
    combo_count[0][0] = 1  # zero containers make sum 0, so there is one way (the empty set)

    # Process each container once; iterate k and s downwards to avoid reuse
    for c in containers:
        if c > total_liters:
            continue
        for k in range(n, 0, -1):
            for s in range(total_liters, c - 1, -1):
                combo_count[k][s] += combo_count[k - 1][s - c]

    # Find smallest k with any ways to make total_liters and return that count
    for k in range(1, n + 1):
        if combo_count[k][total_liters]:
            return combo_count[k][total_liters]
    return 0

In [225]:
# Performance check
time_b_dp = time_solution(solve_b_dp, input_data)
print(f"This is {1000 * time_b_dp / time_a_dp:.0f}x slower than the dynamic programming approach for part A.")
print(f"However, it's still {time_b_iterative / time_b_dp:.0f}x faster than the combinatorial solution for part B.")

solve_b_dp takes 1.64 ms
This is 26x slower than the dynamic programming approach for part A.
However, it's still 23x faster than the combinatorial solution for part B.


In [226]:
# Submit answer
puzzle.answer_b = solve_b_dp(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine.You have completed Day 17! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
